In [364]:
import os, time, shlex, subprocess

import psutil, GPUtil


def gpuInfo() -> dict:
    gpus = GPUtil.getGPUs()
    if len(gpus) == 0: return {}
    gpu = gpus[0]

    # gpu.id, gpu.name, gpu.uuid
    return {
        "load_percent": round(gpu.load*100, 2),
        "memory_percent": round(gpu.memoryUsed/gpu.memoryTotal*100, 2),
        "temperature": gpu.temperature,
    }


class ProcX:
    cmd = []
    wd = ""
    n = 0
    process = None

    __wait_seconds = 5;
    __cpu_interval = 1;

    def __init__(self, cmd: str):
        x = shlex.split(cmd)
        if len(x) < 2: return False
        self.cmd, self.wd = x, os.getcwd()

    def run0(self, excludes=[]) -> bool:
        try:
            process = subprocess.Popen(
              self.cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            )
            # process.kill()
        except:
            print("command start failed: 1")
            return False

        time.sleep(self.__wait_seconds)
        procs = []
        for proc in psutil.process_iter(): # can't get commandline process id directly
            if len(proc.children()) > 0 or proc.pid in excludes: continue
            if proc.name() == self.cmd[1]: procs.append(proc)

        if len(procs) == 0:
            print("command start failed: 2")
            return False

        procs.sort(key=lambda p: p.create_time(), reverse=True)
        # print(procs)
        self.process, self.n = procs[0], 1
        return True

    def run(self) -> bool:
        try:
            process = subprocess.Popen(
              self.cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            )
            # process.kill()
        except:
            print("command start failed 1: ")
            return False

        time.sleep(self.__wait_seconds)
        try:
            self.process = psutil.Process(process.children()[0].children()[0])
        except:
            print("command start failed: 2")
            return False

        print("program pid:", self.process.pid)
        self.n = 1
        return True

    def isRunning(self) -> bool:
        if self.process is None: return False

        return self.process.is_running()

    def kill(self):
        if self.process is None: return

        self.process.kill()

    def status(self) -> dict:
        if self.process is None: return {}

        cpu_percent = round(self.process.cpu_percent(self.__cpu_interval)/psutil.cpu_count(), 2)
        status = self.process.status()
        status = "suspened" if status == "stopped" else status

        return {
            "staus": status,
            "memory_percent": round(self.process.memory_percent(), 2),
            "cpu_percent": cpu_percent,
            "system_gpu_info": gpuInfo(),
        }

In [365]:
os.getcwd()
os.chdir("D:\ShiBanQiao3-24")

cmd = "ShiBanQiao.exe -AudioMixer -PixelStreamingIP=47.117.115.165 -PixelStreamingPort=8204 -RenderOffScreen"
px = ProcX(cmd)

ok = px.run()
print(ok)

command start failed: 2
False


In [349]:
# px.process
px.isRunning()

True

In [350]:
px.status()

{'staus': 'running',
 'memory_percent': 8.7,
 'cpu_percent': 23.45,
 'system_gpu_info': {'load_percent': 93.0,
  'memory_percent': 84.25,
  'temperature': 72.0}}

In [351]:
px.status()

{'staus': 'running',
 'memory_percent': 8.67,
 'cpu_percent': 25.2,
 'system_gpu_info': {'load_percent': 92.0,
  'memory_percent': 84.25,
  'temperature': 72.0}}

In [357]:
px.process.suspend()

In [358]:
px.status()

{'staus': 'suspened',
 'memory_percent': 21.06,
 'cpu_percent': 0.0,
 'system_gpu_info': {'load_percent': 0.0,
  'memory_percent': 90.45,
  'temperature': 61.0}}

In [359]:
px.process.resume()

In [330]:
px.status()

{'staus': 'running',
 'memory_percent': 21.39,
 'cpu_percent': 22.46,
 'system_gpu_info': {'name': 'GeForce GTX 970',
  'load_percent': 92.0,
  'memory_percent': 82.1,
  'temperature': 65.0,
  'uuid': 'GPU-c2a46ebb-7689-71f9-0239-9c1e394d1244'}}

In [346]:
# px.kill()
px.process.terminate()

In [338]:
px.isRunning()

False

In [333]:
px.run()

True

In [335]:
px.status()

{'staus': 'running',
 'memory_percent': 22.46,
 'cpu_percent': 21.16,
 'system_gpu_info': {'name': 'GeForce GTX 970',
  'load_percent': 92.0,
  'memory_percent': 78.34,
  'temperature': 75.0,
  'uuid': 'GPU-c2a46ebb-7689-71f9-0239-9c1e394d1244'}}

In [ ]:
psutil.cpu_t